In [2]:
from pynini import *
from string import punctuation
import tqdm

In [5]:
# 0: 0, <SPACE>
# 1: 1, ASCII punctuation characters (`string.punctuation`)
# 2: 2, a, b, c
# 3: 3, d, e, f
# 4: 4, g, h, i
# 5: 5, j, k, l
# 6: 6, m, n, o
# 7: 7, p, q, r, s
# 8: 8, t, u, v
# 9: 9, w, x, y, z

# create decimal codepoints file for LM

In [124]:
with open('/Users/zhaomengxuan/language_tech/training-monolingual/news.2009.en.shuffled') as file:
    with open('/Users/zhaomengxuan/language_tech/MP1/chr_lm.txt', 'w') as output:
        content = file.readlines()
        for sent in tqdm.tqdm(content[:5000]):
            output.write(' '.join([str(ord(ch)) for ch in sent]))
            output.write('\n')

100%|██████████| 5000/5000 [00:00<00:00, 17425.60it/s]


# ncoder & decoder

In [105]:
decode = string_file('t9.tsv').closure()
encode = invert(decode)

In [112]:
compose('help', encode).stringify()

'4357'

In [125]:
lm = Fst.read("t9.lm")

In [137]:
def encoder(text):
    return compose(text.lower(), encode).stringify()

def decoder(numbers, n):
    lattice = ((numbers @ decode).project(True) @ lm).optimize()
    n_best_results = shortestpath(lattice, nshortest=n, unique=True).paths()
    return [item[1] for item in n_best_results]

# demo

In [136]:
t = "HE ATE SOMETHING"
encoder(t)

'4302830766384464'

In [138]:
decoder('4302830766384464',5)

['he ave something',
 'he aud something',
 'id ate something',
 '43 ave something',
 'ge ave something']

In [139]:
t2 = 'This is a beautiful day.'
encoder(t2)

'844704702023288438503291'

In [140]:
decoder('844704702023288438503291', 10)

['this is a beautiful day-',
 'this is a beautiful day)',
 'this is a beautiful day!',
 'this is a beautiful day:',
 "this is a beautiful day'",
 '8his is a beautiful day-',
 'this is a beautiful 329/',
 'this is a beautiful 3291',
 'this is a beautiful fax-',
 'this is a beautiful fax@']